# FUNCION PARA EL CLUSTERING

In [47]:
import pickle
import pandas as pd
import re

Carga del modelo:

In [48]:
with open('kmode.pkl','rb') as km:
    kmode= pickle.load(km)

Carga de los datos de animales:

In [49]:
adoptados= pd.read_csv("./poblacion_adoptados_v3.csv")

In [50]:
adoptados.columns

Index(['id_animal', 'id_centro', 'Raza', 'Especie', 'Sexo', 'Tamaño',
       'Microchip', 'Tiempo', 'Edad'],
      dtype='object')

Carga de los datos de centros:

In [51]:
centros= pd.read_csv("./poblacion_centros_v3.csv")    

Ejemplo de entrada de usuario:

In [52]:
especie= 'Canina'
raza= 'mestizo'
sexo= 'macho'
tamano= 'grande'
microchip= 'false'
years_tiempo= '1'
months_tiempo= '3'
years_edad= '4'
months_edad= '2'

In [53]:
dataframe= adoptados

In [54]:
# Juntar meses y años en una variable (mismo formato)

def combinar_edad(anios, meses):
    anios_str = str(anios) + " año" + ("s" if anios != '1' else "")
    meses_str = str(meses) + " mes" + ("es" if meses != '1' else "")
    
    if anios != 0 and meses != 0:
        return anios_str + " y " + meses_str
    elif anios != 0:
        return anios_str
    elif meses != 0:
        return meses_str
    else:
        return "0 meses"  # Si ambos son cero, retorna "0 meses"

In [55]:
tiempo_combinado= combinar_edad(years_tiempo, months_tiempo)
años_combinado= combinar_edad(years_edad, months_edad)

In [56]:
dataframe.columns

Index(['id_animal', 'id_centro', 'Raza', 'Especie', 'Sexo', 'Tamaño',
       'Microchip', 'Tiempo', 'Edad'],
      dtype='object')

In [57]:
user_input = {'id_animal':str(len(dataframe) + 1), 'id_centro':'', 'Raza':raza, 'Especie':especie,'Sexo': sexo,
              'Tamaño':tamano, 'Microchip':bool(microchip), 'Tiempo':tiempo_combinado, 'Edad':años_combinado}


In [58]:
# Añadir el nuevo registro al final del DataFrame
dataframe_completo = dataframe.append(user_input, ignore_index=True)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_10708\3280372746.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_completo = dataframe.append(user_input, ignore_index=True)


In [59]:
dataframe_completo.tail()

,id_animal,id_centro,Raza,Especie,Sexo,Tamaño,Microchip,Tiempo,Edad
499996,499997,156,Mestizo,Canina,Hembra,Mediano,True,1 año y 3 meses,2 años y 7 meses
499997,499998,132,Mestizo,Canina,Hembra,Mediano,True,0 año y 0 mes,9 años y 9 meses
499998,499999,826,Mestizo,Canina,Macho,Mediano,True,1 año y 6 meses,10 años y 10 meses
499999,500000,815,Pastor Vasco,Canina,Macho,Pequeño,True,0 año y 1 mes,9 años y 11 meses
500000,500001,,mestizo,Canina,macho,grande,True,1 año y 3 meses,4 años y 2 meses


In [60]:
# Parseo
def parse_age(age_str):
    match = re.search(r'(\d+) años? (y (\d+) meses?)?', age_str)
    if match:
        years_str = match.group(1)
        years = int(years_str) if years_str else 0
        months_str = match.group(3)
        months = int(months_str) if months_str else 0
        total_months = years * 12 + months
        return total_months / 12
    else:
        return None

In [61]:
dataframe_completo['Tiempo_parseado'] = dataframe_completo['Tiempo'].apply(parse_age)
dataframe_completo['Edad_parseado'] = dataframe_completo['Edad'].apply(parse_age)

In [62]:
# Seleccion de columnas para el modelo
dataframe_modelo= dataframe_completo[['Raza', 'Especie', 'Sexo', 'Tamaño',
                                          'Microchip', 'Tiempo_parseado', 'Edad_parseado']]

In [63]:
clusters= kmode.fit_predict(dataframe_modelo)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 51738, cost: 1014827.0


In [64]:
# Pero la columna la añado al conjunto de datos con la columna id
dataframe_completo.insert(0, "Cluster", clusters , True)

In [65]:
# ¿A qué cluster pertenece la última fila del dataframe?
numCluster= dataframe_completo.iloc[-1]['Cluster']

In [66]:
# Borro la última fila del dataframe para trabajar con el resto de datos
dataframe_completo= dataframe_completo[:-1]

In [67]:
# De los datos iniciales, ¿Cuales pertenecen al mismo cluster?
mascara = dataframe_completo.loc[:, 'Cluster'] == numCluster
mismoCluster= dataframe_completo[mascara]

In [68]:
# Del mismo cluster, ¿Qué número de registros hay para cada centro?
frecuencia_serie= mismoCluster.groupby('id_centro').size()
sorted_serie = frecuencia_serie.sort_values(ascending= False)  

In [75]:
sorted_serie

id_centro
743    1095
650     999
715     990
899     973
797     948
       ... 
166      11
829      10
224       9
189       8
481       8
Length: 951, dtype: int64

In [82]:
sorted_df = pd.DataFrame(sorted_serie, columns=['frecuencia'])

In [84]:
# Me queda ver con que centro se corresponde cada id
df_ok= pd.merge(sorted_df,centros, on= 'id_centro')

In [85]:
df_ok

,id_centro,frecuencia,Nombre,Direccion,CCAA,Latitud,Longitud,Telefono,URL,Correo,Facebook,Categoria,Tamaño,Capacidad,Ocupacion
0,743,1095,RESCATE ANIMAL MARINA ALTA (R.A.M.A),"PLAÇA DE BAIX, 5 Xàbia/Jávea ALICANTE Valencia",Valencia,38.788958,0.163724,+34887 95 64 28,http://www.rescateanimalmarinaaltar.a.m.a.com,resca@porras.es,login/rescateanimalmarinaaltar.a.m.a,canina,grande,499,892
1,650,999,Associació Progat Terrassa,"Carrer Goleta, 11 8221 Terrassa Barcelona Vall...",Cataluña,41.562979,2.008178,+34 807933070,http://www.associaciprogatterrassa.com,associacip@arnaiz.com,faq/associaciprogatterrassa,felina,grande,478,867
2,715,990,ANIMALS SENSE SOSTRE D'ELX,"JOAQUIN GARCIA MORA, 58 Elx/Elche ALICANTE Val...",Valencia,38.275135,-0.712642,+34883 192 892,http://www.animalssensesostredelx.com,anima@ballester.es,terms/animalssensesostredelx,mixta,grande,464,830
3,899,973,ASOCIACION CRUZ AZUL PARA MASCOTAS,"C/ Bernal nº13, SAN JAVIER, Murcia, España",Murcia,37.799818,-0.806589,+34 848 75 82 38,http://www.asociacioncruzazulparamascotas.com,asociacio@calzada.com,main/asociacioncruzazulparamascotas,mixta,grande,490,814
4,797,948,Asociación Protectora de Animales Miaumor nan,nan nan nan nan nan Euskadi,Euskadi,51.921559,4.482812,+34946 013 304,http://www.asociacinprotectoradeanimalesmiaumo...,asocia@hidalgo.net,terms/asociacinprotectoradeanimalesmiaumornan,mixta,grande,487,855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,166,11,ALMA FELINA ALMERIA,"C/ LABRADORES, 16 , Almería (ALMERIA) , Andalu...",Andalucía,36.848279,-2.435888,+34925 536 833,http://www.almafelinaalmeria.com,almafe@rivera-codina.es,faq/almafelinaalmeria,felina,pequeña,30,10
947,829,10,ASOCIACIÓN VILLAMICHINOS COLONIAS FELINAS,C/ Carmelo Tecedor nº 7 bajo,La Rioja,42.424735,-2.421250,+34 982 052 964,http://www.asociacinvillamichinoscoloniasfelin...,asocia@reina.org,terms/asociacinvillamichinoscoloniasfelinas,felina,pequeña,28,12
948,224,9,CANINA COMARCA DEL MARMOL A.C.C.M.,"AVDA. ALMERIA, 3 , Macael (ALMERIA) , Andalucí...",Andalucía,37.334287,-2.302227,+34803 648 468,http://www.caninacomarcadelmarmola.c.c.m..com,caninaco@catala-guillen.com,register/caninacomarcadelmarmola.c.c.m.,canina,pequeña,33,10
949,189,8,ADOPTA GRANADA,"C/ LOS ALAYOS, Nº 11 , Ogíjares (GRANADA) , An...",Andalucía,37.112111,-3.615597,+34944 60 19 47,http://www.adoptagranada.com,adopta@fuster.es,about/adoptagranada,mixta,pequeña,20,5


df_ok son los datos ordenados en funcion de las coincidencias, es decir, lo que tengo que visualizar en la aplicación de forma bonita

In [30]:
def funcion(dataframe, user_input):
    # Importo el dataset centros (lo voy a necesitar localmente)
    centros= pd.read_csv("./poblacion_centros_v3.csv")    
    
    # Introduzco el registro proporcionado por el usuario en la última posición
    user_input.append(len(dataframe) + 1)  # Agregar el ID del animal
    dataframe_completo = dataframe + [user_input]  
    
    # Ahora aplico el modelo al df sin las columnas id
    dataframe_modelo= dataframe_completo[['Raza', 'Especie', 'Sexo', 'Tamaño',
                                          'Microchip', 'Tiempo', 'Edad']]
    
    clusters= kmode.fit_predict(dataframe_modelo)
    
    # Pero la columna la añado al conjunto de datos con la columna id
    dataframe_completo.insert(0, "Cluster", clusters , True)
    
    # ¿A qué cluster pertenece la última fila del dataframe?
    numCluster= dataframe_completo.iloc[-1]['Cluster']
    
    # Saco la última fila del dataframe para trabajar con el resto de datos
    dataframe_completo= dataframe_completo[:-1]
    
    # De los datos iniciales, ¿Cuales pertenecen al mismo cluster?
    mascara = dataframe_completo.loc[:, 'Cluster'] == numCluster
    mismoCluster= dataframe_completo[mascara]
    
    # Del mismo cluster, ¿Qué número de registros hay para cada centro?
    frecuencia_serie= mismoCluster.groupby('id_centro').size()
    sorted_serie = frecuencia_serie.sort_values(ascending= False) 
    
    # Selecciono los 5 primeros centros para hacer un top 5
    primeros = sorted_serie.iloc[:5]
    
    # Paso la serie a df y añado una columna "top" (+ algunas transformaciones)
    primeros_df = pd.DataFrame(primeros, columns=['frecuencia'])
    top = [1,2,3,4,5] 
    primeros_df['top'] = top

    primeros_df_ok = pd.DataFrame(primeros_df,index=primeros_df.index).rename_axis('id_centro')
    df_ok= primeros_df_ok.reset_index() # Paso el indice por defecto a columna
    
    # Me queda ver con que centro se corresponde cada id
    top5= pd.merge(df_ok,centros, on= 'id_centro')
    
    #top5= top5.drop(['Unnamed: 0','id_centro', 'frecuencia','descripcion', 'imagen_logo','imagen_fondo', 'google_maps'], axis=1) 
    
    return top5

In [5]:
centros= pd.read_csv("./poblacion_centros_v3.csv") 

In [6]:
centros

,Nombre,Direccion,CCAA,Latitud,Longitud,Telefono,URL,Correo,Facebook,Categoria,Tamaño,Capacidad,Ocupacion,id_centro
0,HELPING ANIMALS ALMANZORA,"C/ HUELVA Nº 5 , Arboleas (ALMERIA) , Andalucí...",Andalucía,37.349970,-2.073728,+34877 393 789,http://www.helpinganimalsalmanzora.com,helpinga@reguera.com,register/helpinganimalsalmanzora,mixta,mediana,253,252,1
1,PROTECTORA DE ANIMALES ROCIANA ACOGE,"CL VENDIMIADORES 33 , Rociana del Condado (HUE...",Andalucía,37.312134,-6.595455,+34823 95 92 00,http://www.protectoradeanimalesrocianaacoge.com,prote@castaneda.net,author/protectoradeanimalesrocianaacoge,mixta,mediana,151,104,2
2,ABRAZO ANIMAL JAMILENA,"C/ MORERIA, 27 , Martos (JAEN) , Andalucía, Es...",Andalucía,37.724364,-3.968021,+34 943107598,http://www.abrazoanimaljamilena.com,abraz@valls-arrieta.es,index/abrazoanimaljamilena,mixta,mediana,207,214,3
3,ACTION FOR ANIMALS,"AVENIDA DE LA CONSTITUCION Nº 3 , Cómpeta (MAL...",Andalucía,36.833335,-3.976101,+34 855446257,http://www.actionforanimals.com,actio@aliaga-heredia.com,main/actionforanimals,felina,pequeña,94,167,4
4,ADA UMBRETE - DEFENSA ANIMAL,"C/ VOLUNTAD, Nº 2 , Umbrete (SEVILLA) , Andalu...",Andalucía,37.376780,-6.158583,+34982 217 289,http://www.adaumbretedefensaanimal.com,adaumbret@alcala.net,login/adaumbretedefensaanimal,mixta,grande,434,396,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,CER MURCHANTE,"Calle Plana Meneses, Nº 25, MURCHANTE, España",Navarra,42.031125,-1.663485,+34 749031317,http://www.cermurchante.com,cermurcha@portero.es,search/cermurchante,mixta,mediana,232,66,947
947,KATZONDINA HUNTERS,"Calle Santa Agueda, nº 7, JAURRIETA, España",Navarra,42.886857,-1.137845,+34 703 86 89 90,http://www.katzondinahunters.com,katzondin@gras-jara.org,search/katzondinahunters,mixta,mediana,134,66,948
948,OBANOS BIZI,"Plaza de los Fueros, Nº 1, OBANOS, España",Navarra,42.679362,-1.785972,+34 926 48 89 83,http://www.obanosbizi.com,obanos@pujol-villalobos.es,main/obanosbizi,mixta,pequeña,75,56,949
949,ACACONBAR,"Calle Auroros, 65, TUDELA, España",Navarra,42.063825,-1.620283,+34 922 581 454,http://www.acaconbar.com,acacon@elias.es,login/acaconbar,mixta,mediana,196,146,950


Parámetros de usuario:

In [16]:
adoptados

,id_animal,id_centro,Raza,Especie,Sexo,Tamaño,Microchip,Tiempo,Edad
0,1,543,Mestizo,Canina,Hembra,Mediano,False,0 año y 3 meses,3 años y 1 mes
1,2,118,Mestizo,Canina,Macho,Pequeño,True,0 año y 2 meses,3 años y 7 meses
2,3,757,Mestizo,Canina,Hembra,Pequeño,True,0 año y 0 mes,11 años y 6 meses
3,4,609,Pastor Belga,Canina,Hembra,Mediano,False,0 año y 4 meses,1 año y 0 mes
4,5,743,Mestizo,Canina,Hembra,Mediano,True,0 año y 0 mes,3 años y 10 meses
...,...,...,...,...,...,...,...,...,...
499995,499996,756,Mestizo,Canina,Macho,Mediano,True,0 año y 0 mes,9 años y 2 meses
499996,499997,156,Mestizo,Canina,Hembra,Mediano,True,1 año y 3 meses,2 años y 7 meses
499997,499998,132,Mestizo,Canina,Hembra,Mediano,True,0 año y 0 mes,9 años y 9 meses
499998,499999,826,Mestizo,Canina,Macho,Mediano,True,1 año y 6 meses,10 años y 10 meses


In [17]:
Raza= 'Pastor Vasco'
Especie= 'Canina'
Sexo= 'Hembra'
Tamaño= 'Pequeño'
Microchip= False
Tiempo= '2 años y 5 meses'
Edad= '3 años y 4 meses'

In [20]:
# Parseo de la edad y tiempo

def parse_age(age_str):
    match = re.search(r'(\d+) años? (y (\d+) meses?)?', age_str)
    if match:
        years_str = match.group(1)
        years = int(years_str) if years_str else 0
        months_str = match.group(3)
        months = int(months_str) if months_str else 0
        total_months = years * 12 + months
        return total_months / 12
    else:
        return None

In [22]:
user_input

['Pastor Vasco',
 'Canina',
 'Hembra',
 'Pequeño',
 False,
 2.4166666666666665,
 3.3333333333333335]

In [23]:
user_input = [Raza, Especie, Sexo, Tamaño, Microchip, parse_age(Tiempo), parse_age(Edad)]

Aplico función a los parámetros del usuario:

In [31]:
output= funcion(dataframe, user_input)

ValueError: Unable to coerce to Series, length must be 9: given 1

In [32]:
centros= pd.read_csv("./poblacion_centros_v3.csv") 

In [33]:
dataframe= adoptados

In [34]:
dataframe

,id_animal,id_centro,Raza,Especie,Sexo,Tamaño,Microchip,Tiempo,Edad
0,1,543,Mestizo,Canina,Hembra,Mediano,False,0 año y 3 meses,3 años y 1 mes
1,2,118,Mestizo,Canina,Macho,Pequeño,True,0 año y 2 meses,3 años y 7 meses
2,3,757,Mestizo,Canina,Hembra,Pequeño,True,0 año y 0 mes,11 años y 6 meses
3,4,609,Pastor Belga,Canina,Hembra,Mediano,False,0 año y 4 meses,1 año y 0 mes
4,5,743,Mestizo,Canina,Hembra,Mediano,True,0 año y 0 mes,3 años y 10 meses
...,...,...,...,...,...,...,...,...,...
499995,499996,756,Mestizo,Canina,Macho,Mediano,True,0 año y 0 mes,9 años y 2 meses
499996,499997,156,Mestizo,Canina,Hembra,Mediano,True,1 año y 3 meses,2 años y 7 meses
499997,499998,132,Mestizo,Canina,Hembra,Mediano,True,0 año y 0 mes,9 años y 9 meses
499998,499999,826,Mestizo,Canina,Macho,Mediano,True,1 año y 6 meses,10 años y 10 meses


In [35]:
user_input = []

['Pastor Vasco',
 'Canina',
 'Hembra',
 'Pequeño',
 False,
 2.4166666666666665,
 3.3333333333333335,
 500001]

In [ ]:
def funcion(dataframe, user_input):
    # Importo el dataset centros (lo voy a necesitar localmente)
    centros= pd.read_csv("./poblacion_centros_v3.csv")    
    
    # Introduzco el registro proporcionado por el usuario en la última posición
    user_input.append(len(dataframe) + 1)  # Agregar el ID del animal
    dataframe_completo = dataframe + [user_input]  
    
    # Ahora aplico el modelo al df sin las columnas id
    dataframe_modelo= dataframe_completo[['Raza', 'Especie', 'Sexo', 'Tamaño',
                                          'Microchip', 'Tiempo', 'Edad']]
    
    clusters= kmode.fit_predict(dataframe_modelo)
    
    # Pero la columna la añado al conjunto de datos con la columna id
    dataframe_completo.insert(0, "Cluster", clusters , True)
    
    # ¿A qué cluster pertenece la última fila del dataframe?
    numCluster= dataframe_completo.iloc[-1]['Cluster']
    
    # Saco la última fila del dataframe para trabajar con el resto de datos
    dataframe_completo= dataframe_completo[:-1]
    
    # De los datos iniciales, ¿Cuales pertenecen al mismo cluster?
    mascara = dataframe_completo.loc[:, 'Cluster'] == numCluster
    mismoCluster= dataframe_completo[mascara]
    
    # Del mismo cluster, ¿Qué número de registros hay para cada centro?
    frecuencia_serie= mismoCluster.groupby('id_centro').size()
    sorted_serie = frecuencia_serie.sort_values(ascending= False) 
    
    # Selecciono los 5 primeros centros para hacer un top 5
    primeros = sorted_serie.iloc[:5]
    
    # Paso la serie a df y añado una columna "top" (+ algunas transformaciones)
    primeros_df = pd.DataFrame(primeros, columns=['frecuencia'])
    top = [1,2,3,4,5] 
    primeros_df['top'] = top

    primeros_df_ok = pd.DataFrame(primeros_df,index=primeros_df.index).rename_axis('id_centro')
    df_ok= primeros_df_ok.reset_index() # Paso el indice por defecto a columna
    
    # Me queda ver con que centro se corresponde cada id
    top5= pd.merge(df_ok,centros, on= 'id_centro')
    
    #top5= top5.drop(['Unnamed: 0','id_centro', 'frecuencia','descripcion', 'imagen_logo','imagen_fondo', 'google_maps'], axis=1) 
    
    return top5